In [141]:
import plotly.express as px
import json 
import cufflinks as cf
import pandas as pd
import seaborn as sns
import chart_studio.plotly as py
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot

init_notebook_mode(connected=True)
cf.go_offline()


with open ('datos.json','r')as gr:
    dt=gr.read()
granma=json.loads(dt)

print(granma["municipios"]["Guisa"])


{'residentesH_M_14-20': [[24868, 24656, 24448, 24183, 23994, 23690, 23517], [23283, 23121, 22927, 22696, 22462, 22192, 22002]], 'SMI-19-22': [-555, -359, -243], 'TMI-19-22': [-12.0, -7.9, -5.0], 'SME-19-22': [-22, -30, -1], 'TME-19-22': [-0.5, -0.66, 0.0]}
